<a href="https://colab.research.google.com/github/YG15/DataHack2019/blob/master/yonathan_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !wget https://raw.githubusercontent.com/YG15/DataHack2019/master/src/functions/get_network_hierarchy.py
# from get_network_hierarchy import *

In [0]:
def diff_list(li1, li2): 
    return (list(set(li1) - set(li2))) 

def get_categ_col_dummy (df):    
    numerical_columns = df.describe().columns
    categorical_cols=diff_list(df.columns,numerical_columns)
    
    categorial = df[categorical_cols]
    #to prevent case-sensitive separation to different categories:
    categorial=categorial.apply(lambda x: x.str.lower(), axis=1)
    categorial_dummies = pd.get_dummies(categorial)
    df.drop(columns=categorical_cols, inplace=True)
    df=df.join(categorial_dummies)
    return df

In [4]:
url = 'https://armis-datahack.s3.amazonaws.com/all_devices.csv'
devices_df = pd.read_csv(url, index_col = 0)
devices_df.head()

,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version
113,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9
587,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9
668,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0
830,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN
886,0,1755023,TABLET,iPad,Apple,iOS,NaN


In [5]:
keep_os = ['Android', 'iOS', 'Mac OS X', 'Windows', 'watchOS',
       'Watch OS', 'Tizen', 'Symbian', 'Miui OS', 'Linux',
       'Axis Firmware', 'Firefox OS', 'Cisco IOS-XE', 'Cisco IOS',
       'Debian', 'Link-OS', 'Cisco NX-OS']
devices_df['operating_system'] = [ val if val in keep_os else 'OTHER' for val in devices_df.operating_system]
devices_df.drop(columns =['operating_system_version','model','manufacturer'], inplace=True)
devices_df = get_categ_col_dummy(devices_df)
devices_df.head()

,network_id,device_id,type_ip_camera,type_mobile_phone,type_pc,type_printer,type_tablet,type_voip,type_watch,operating_system_android,operating_system_axis firmware,operating_system_cisco ios,operating_system_cisco ios-xe,operating_system_cisco nx-os,operating_system_debian,operating_system_firefox os,operating_system_ios,operating_system_link-os,operating_system_linux,operating_system_mac os x,operating_system_miui os,operating_system_other,operating_system_symbian,operating_system_tizen,operating_system_watch os,operating_system_watchos,operating_system_windows
113,0,1028623,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
587,0,48047,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
668,0,123568,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
830,0,95366,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
886,0,1755023,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
sessions_df = pd.read_csv(r'drive/My Drive/all_sessions.csv', index_col = 0)
sessions_df.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev
0,0,35,1565074800,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,49152,TCP,790889.0,260,21855,7815,0.067503,67,NaN,39,235,25,930,465,560.384615,NaN,190.219349,1563,0,200.384615,NaN,529.370014,93.000000,93.000000,93.000000,93.000000,0.000000,312.6,312.6,312.6,312.6,0.0
1,0,35,1565053200,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,49153,TCP,1604622.0,178,8326,28656,0.000000,16,NaN,18,88,90,467,427,462.555556,467.0,12.935233,1592,1592,1592.000000,1592.0,0.000000,106.750000,93.400000,94.883333,93.000000,4.317134,318.4,318.4,318.4,318.0,0.0
2,0,57,1565082000,e16257c983f2c35d41f39d425651972fa1905e46e968d7...,d43dad76e6cef2231d2efc743e498996b40f8b13fc120b...,443,TCP,NaN,67,4013,0,0.000000,40,NaN,7,67,0,3521,82,573.285714,NaN,1299.819823,0,0,0.000000,NaN,0.000000,64.018182,41.000000,44.288312,41.000000,8.700055,NaN,NaN,NaN,NaN,NaN
3,0,57,1565082000,1a4f860269acca6c264f00d84c4b63aad00b8f93a77250...,945e37dab8aee93dd4e650f8d17d76a3adfbc6aa70ebba...,443,TCP,NaN,45,9158,0,0.000000,3,NaN,4,45,0,3401,935,2289.500000,2411.0,1018.074490,0,0,0.000000,0.0,0.000000,226.733333,133.571429,194.968236,209.000000,41.777266,NaN,NaN,NaN,NaN,NaN
4,0,57,1565082000,df106cbe1ba4a700c00ec8883490f40a8afdb75c15a9ea...,4b43e85e630c2d18a0afaa2a6366367c4fc52d32b4ba5b...,443,TCP,NaN,9,1671,0,0.000000,0,NaN,1,9,0,1671,1671,1671.000000,NaN,NaN,0,0,0.000000,NaN,NaN,185.666667,185.666667,185.666667,185.666667,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# url = 'https://armis-datahack.s3.amazonaws.com/all_sessions.csv'
# sessions_df = pd.read_csv(url)
# sessions_df['real_device'] = [float(str(id)+'.'+str(net)) for id,net in zip (sessions_df.device_id,sessions_df.network_id)]
# sessions_df.drop(columns =['device_id'], inplace=True)
# sessions_df.head()

In [0]:
#sessions_df = get_network_hierarchy(sessions_df)

In [0]:
numeric_col = ['packets_count',
       'outbound_bytes_count', 'inbound_bytes_count', 'packet_loss',
       'retransmit_count', 'latency', 'session_count',
       'outbound_packets_count', 'inbound_packets_count', 'outbound_bytes_max',
       'outbound_bytes_min', 'outbound_bytes_mean', 'outbound_bytes_median',
       'outbound_bytes_stddev', 'inbound_bytes_max', 'inbound_bytes_min',
       'inbound_bytes_mean', 'inbound_bytes_median', 'inbound_bytes_stddev',
       'outbound_packet_size_max', 'outbound_packet_size_min',
       'outbound_packet_size_mean', 'outbound_packet_size_median',
       'outbound_packet_size_stddev', 'inbound_packet_size_max',
       'inbound_packet_size_min', 'inbound_packet_size_mean',
       'inbound_packet_size_median', 'inbound_packet_size_stddev',]
for c in numeric_col:
  if sessions_df[c].dtype !='object':
    sessions_df[c] = np.log10(sessions_df[c]+1)
sessions_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app


In [0]:
def range_list(val_list):
    min_val = min(val_list)
    max_val = max(val_list)

    return (max_val-min_val)
  
classic_functions = [np.nanmin,np.nanmax,np.nanmean,np.nanstd, np.nanmedian]

df_gb = sessions_df.groupby(['network_id','device_id'],as_index=False).agg({'host':'nunique',
                                                      'host_ip':'nunique',
                                                      'port_dst':'nunique',
                                                      'timestamp':['nunique','count',('range', lambda x: range_list(x))],
                                                      'transport_protocol':'nunique',
                                                      'packets_count':classic_functions,
                                                      'outbound_bytes_count':classic_functions, 
                                                      'inbound_bytes_count':classic_functions, 
                                                      'packet_loss':classic_functions,
                                                      'retransmit_count':classic_functions, 
                                                      'latency':classic_functions, 
                                                      'session_count':classic_functions,
                                                      'outbound_packets_count':classic_functions, 
                                                      'inbound_packets_count':classic_functions, 
                                                      'outbound_bytes_max':classic_functions,
                                                      'outbound_bytes_min':classic_functions, 
                                                      'outbound_bytes_mean':classic_functions, 
                                                      'outbound_bytes_median':classic_functions,
                                                      'outbound_bytes_stddev':classic_functions, 
                                                      'inbound_bytes_max':classic_functions, 
                                                      'inbound_bytes_min':classic_functions,
                                                      'inbound_bytes_mean':classic_functions, 
                                                      'inbound_bytes_median':classic_functions, 
                                                      'inbound_bytes_stddev':classic_functions,
                                                      'outbound_packet_size_max':classic_functions, 
                                                      'outbound_packet_size_min':classic_functions,
                                                      'outbound_packet_size_mean':classic_functions, 
                                                      'outbound_packet_size_median':classic_functions,
                                                      'outbound_packet_size_stddev':classic_functions, 
                                                      'inbound_packet_size_max':classic_functions,
                                                      'inbound_packet_size_min':classic_functions, 
                                                      'inbound_packet_size_mean':classic_functions,
                                                      'inbound_packet_size_median':classic_functions, 
                                                      'inbound_packet_size_stddev':classic_functions})

df_gb.columns = ["".join(x) for x in df_gb.columns.ravel()]

In [0]:
final_df = pd.merge(devices_df, df_gb, how = 'right', on=['network_id','device_id'])
print(devices_df.shape, df_gb.shape, final_df.shape)

In [0]:
final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna(0)

In [0]:
# final_df.to_csv('features_v3.csv')

In [0]:
# from google.colab import files
# files.download("features_v3.csv")

In [0]:
from google.colab import drive
drive.mount('drive')
final_df.to_csv('features_v3.csv')
!cp features_v3.csv drive/My\ Drive/

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
def run_IF(df):
    model = IsolationForest(behaviour = 'new', max_samples=100, 
                          random_state = 1, contamination = 0)
    model.fit(df)
    anomaly_scores = model.decision_function(df)
    
    return anomaly_scores


df_features = final_df

X = df_features.copy()
X.drop(['device_id'], inplace=True, axis=1)

# Run model
anomaly_scores = run_IF(X)
anomaly_scores = -anomaly_scores

# Save results
results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : anomaly_scores})

In [0]:
#%% Submission

arr_to_submit = results.to_json(orient='values')

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "The Whale and the Petunias"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))